## Importación de librerías

In [61]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

## Configuración de variables globales

In [62]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
with open('labels.json', 'r') as f:
    LABELS_JSON = json.load(f) 
OUTPUT_CSV = SETUP_JSON['dataset_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['dataset_parquet'] # Output CSV file

## Cargar el dataset

In [63]:
df = pd.read_csv(OUTPUT_CSV) # Leer csv

## Extracción de secuencias de paquetes

Obtener todas las secuencias de paquetes presentes en el dataset.

In [64]:
# Crear una copia del DataFrame original para no modificarlo directamente.
df_temp = df.copy()

# Cada vez que el valor de 'label' cambia, se incrementa el contador acumulativo.
df_temp['sequence'] = (df_temp['Label'] != df_temp['Label'].shift()).cumsum()

# Agrupar el DataFrame por la columna 'sequence' y crear una lista de DataFrames.
sequences = [sequence for _, sequence in df_temp.groupby('sequence')]  

for sequence_df in sequences:  # Iterar sobre cada DataFrame en la lista de secuencias.
    # Eliminar la columna 'sequence' de cada DataFrame, ya que solo se usó para agrupar.
    sequence_df.drop(columns='sequence', inplace=True)  
    # Reiniciar los índices de cada DataFrame para que comiencen desde 0.
    sequence_df.reset_index(drop=True, inplace=True)

## Análisis de las secuencias

In [65]:
print(f"Total de secuencias de paquetes encontradas: {len(sequences)}")

Total de secuencias de paquetes encontradas: 15947


In [66]:
print(f"Primera secuencia: {sequences[0]['Label'].unique()}")
print(f"Número de paquetes de la primera secuencia: {sequences[0].shape[0]}")

Primera secuencia: [0]
Número de paquetes de la primera secuencia: 94


In [67]:
print(f"Última secuencia: {sequences[-1]['Label'].unique()}")
print(f"Número de paquetes de la última secuencia: {sequences[-1].shape[0]}")

Última secuencia: [10]
Número de paquetes de la última secuencia: 17


In [68]:
sequences_df = pd.DataFrame({
    'Sequence Size': [sequence.shape[0] for sequence in sequences],
    'Sequence Label': [sequence['Label'].unique()[0] for sequence in sequences]
})

In [69]:
sequences_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15947 entries, 0 to 15946
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Sequence Size   15947 non-null  int64
 1   Sequence Label  15947 non-null  int64
dtypes: int64(2)
memory usage: 249.3 KB


In [70]:
sequences_df.head()

,Sequence Size,Sequence Label
0,94,0
1,64,1
2,1,0
3,5610,1
4,1,0


In [71]:
print("Análisis estadístico de las secuencias")
print(f"{sequences_df['Sequence Size'].describe()}")

Análisis estadístico de las secuencias
count     15947.000000
mean         45.026212
std        2050.380944
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max      237376.000000
Name: Sequence Size, dtype: float64


Obtener media y desviación estándar del número de paquetes.

In [72]:
# Obtener la media de paquetes por secuencia
mean = sequences_df['Sequence Size'].mean()

# Obtener la desviación estándar muestral (por defecto ddof=1)
deviation = sequences_df['Sequence Size'].std()

print(f"Número promedio de paquetes por secuencia: {mean}")
print(f"Desviación estándar (muestral): {deviation}")

Número promedio de paquetes por secuencia: 45.02621182667586
Desviación estándar (muestral): 2050.3809442995553


Obtener número de secuencias asociadas a cada etiqueta

In [73]:
sequences_df["Sequence Label"].value_counts()  # Contar la cantidad de secuencias por etiqueta

Sequence Label
2     5026
3     4668
0     3805
10    1548
5      635
8      109
7       73
9       32
4       30
1       18
6        3
Name: count, dtype: int64

## Normalizar las secuencias

In [74]:
# Número de características de cada secuencia
num_features = sequences[0].shape[1] - 2
# Número de secuencias
num_sequences = len(sequences)

Establecer tamaño fijo para todas las secuencias y separar en X e y

In [75]:
# Media de paquetes por secuencia como longitud fija
sequence_length = int(mean)

X_fixed = []
y_fixed = []

for seq in sequences:
    data = seq.drop(columns=['Label', 'Label Name'])
    label = seq['Label'].iloc[0]

    if len(data) < sequence_length:
        # Rellenar con ceros
        pad_width = sequence_length - len(data)
        padded = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    else:
        # Recortar
        padded = data.iloc[:sequence_length]

    X_fixed.append(padded)
    y_fixed.append(label)

X = np.array(X_fixed)
y = np.array(y_fixed)

In [76]:
print(X.shape)
print(y.shape)

(15947, 45, 79)
(15947,)


Escalamiento de características

In [77]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, num_features)).reshape(num_sequences, sequence_length, num_features)

In [78]:
len(X_scaled[0][0])

79

In [79]:
X_scaled[0][0]

array([-1.97766439e-01, -3.05405244e-01,  5.68190209e+00,  8.12004927e+00,
       -4.73690641e-02, -2.50313939e-02, -4.80153164e-02, -2.14519146e-02,
       -8.64026885e-02, -4.50510071e-02, -1.21849657e-01, -1.49213584e-01,
       -1.26366686e-01, -2.17263470e-02, -1.05943549e-01, -1.24705272e-01,
       -1.58717163e-02, -5.66842088e-02,  1.65410922e+01, -8.51779032e-02,
        8.04836710e+00,  2.01800762e+01,  8.12388850e+00,  1.08897615e+01,
       -1.06614661e-01,  8.05041136e+00,  1.11119149e+01, -9.62176715e-02,
       -8.66951591e-02, -8.82799320e-02, -9.11858688e-02, -7.39201631e-02,
       -4.84270760e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -4.80447927e-02, -2.54582743e-02, -5.91369030e-02, -5.31152136e-02,
       -4.36678477e-02, -1.02340459e-01, -1.58660621e-01, -1.58068186e-01,
       -1.15428856e-02, -1.41177517e-02, -4.84270760e-02, -1.69606715e-01,
       -2.33054881e-01, -2.07100081e-01, -2.59252925e-02,  0.00000000e+00,
       -1.69606715e-01, -

In [80]:
X_scaled[-1][-1]

array([-0.19776644, -0.30540524, -0.28998415, -0.14015261, -0.04787582,
       -0.02503139, -0.04801532, -0.02145191, -0.08640269, -0.04505101,
       -0.12184966, -0.14921358, -0.12636669, -0.02172635, -0.10594355,
       -0.12470527, -0.01587172, -0.05668464, -0.10808506, -0.08522507,
       -0.12741051, -0.07621184, -0.13981388, -0.09463694, -0.1067136 ,
       -0.1270681 , -0.07738228, -0.09621767, -0.08669516, -0.08827993,
       -0.09118587, -0.07392016, -0.04842708,  0.        ,  0.        ,
        0.        , -0.04804479, -0.02545827, -0.05913774, -0.05311521,
       -0.04366785, -0.10234046, -0.15866062, -0.15806819, -0.01154289,
       -0.01411775, -0.04842708, -0.16960672, -0.23305488, -0.20710008,
       -0.02592529,  0.        , -0.16960672, -0.17087755, -0.15886076,
       -0.12184966, -0.10594355,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.04787582, -0.04801532,
       -0.02503139, -0.02145191, -0.20488827, -0.04115293, -0.04

## Separar secuencias en entrenamiento y prueba

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [82]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(12757, 45, 79) (12757,)
(3190, 45, 79) (3190,)


Convertir los arrays a tensores

In [83]:
# Convertir a tensores
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [84]:
y_test_tensor

tensor([ 3, 10,  3,  ...,  2,  3,  5])

In [85]:
num_classes = len(sequences_df['Sequence Label'].unique())
num_classes

11

## Inicialización del modelo

In [86]:
# Modelo LSTM
class NetFlowLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)         # h_n shape: (1, batch, hidden)
        return self.classifier(h_n[-1])    # usar último estado oculto

model = NetFlowLSTM(input_dim=num_features, hidden_dim=32, output_dim=num_classes)

## Entrenamiento

In [90]:
# Configurar entrenamiento
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 1000

# Entrenar el modelo
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Evaluar el modelo
    model.eval()
    with torch.no_grad():
        train_preds = torch.argmax(outputs, dim=1)
        train_acc = (train_preds == y_train_tensor).float().mean().item()

        test_outputs = model(X_test_tensor)
        test_preds = torch.argmax(test_outputs, dim=1)
        test_acc = (test_preds == y_test_tensor).float().mean().item()

        # Calcular accuracy por clase en test
        class_acc = {}
        for class_label in torch.unique(y_test_tensor):
            mask = y_test_tensor == class_label
            acc = (test_preds[mask] == y_test_tensor[mask]).float().mean().item()
            class_acc[class_label.item()] = acc

    # Mostrar resultados
    print(f"\nEpoch {epoch+1:02d} | Loss: {loss.item():.4f} | Train Accuracy: {train_acc:.2f} | Test Accuracy: {test_acc:.2f}")
    for cls, acc in class_acc.items():
        print(f"Accuracy Clase {cls} ({LABELS_JSON[str(cls)]}) (Test): {acc:.2f}")
    print("-"*80)


Epoch 01 | Loss: 0.6826 | Train Accuracy: 0.92 | Test Accuracy: 0.91
Accuracy Clase 0 (Benign) (Test): 0.94
Accuracy Clase 1 (FTP-BruteForce) (Test): 0.00
Accuracy Clase 2 (DoS attacks-GoldenEye) (Test): 0.97
Accuracy Clase 3 (DoS attacks-Slowloris) (Test): 0.98
Accuracy Clase 4 (DoS attacks-SlowHTTPTest) (Test): 1.00
Accuracy Clase 5 (DDOS attack-LOIC-UDP) (Test): 0.00
Accuracy Clase 7 (Brute Force -Web) (Test): 0.00
Accuracy Clase 8 (Brute Force -XSS) (Test): 0.00
Accuracy Clase 9 (SQL Injection) (Test): 0.00
Accuracy Clase 10 (Bot) (Test): 0.96
--------------------------------------------------------------------------------

Epoch 02 | Loss: 0.6349 | Train Accuracy: 0.92 | Test Accuracy: 0.88
Accuracy Clase 0 (Benign) (Test): 0.95
Accuracy Clase 1 (FTP-BruteForce) (Test): 0.00
Accuracy Clase 2 (DoS attacks-GoldenEye) (Test): 0.86
Accuracy Clase 3 (DoS attacks-Slowloris) (Test): 0.98
Accuracy Clase 4 (DoS attacks-SlowHTTPTest) (Test): 1.00
Accuracy Clase 5 (DDOS attack-LOIC-UDP) (Te

In [ ]:
"""# Asignar la columna 'ID' dentro de cada DataFrame en train
for i, sequence in enumerate(train["Sequence"]):
    sequence["ID"] = train.iloc[i]["ID"]
# Asignar la columna 'ID' dentro de cada DataFrame en train
for i, sequence in enumerate(test["Sequence"]):
    sequence["ID"] = test.iloc[i]["ID"]


# Concatenar manteniendo la columna 'ID'
train_concatenado = pd.concat(train["Sequence"].tolist(), ignore_index=True)
# Concatenar manteniendo la columna 'ID'
test_concatenado = pd.concat(test["Sequence"].tolist(), ignore_index=True)"""

'# Asignar la columna \'ID\' dentro de cada DataFrame en train\nfor i, sequence in enumerate(train["Sequence"]):\n    sequence["ID"] = train.iloc[i]["ID"]\n# Asignar la columna \'ID\' dentro de cada DataFrame en train\nfor i, sequence in enumerate(test["Sequence"]):\n    sequence["ID"] = test.iloc[i]["ID"]\n\n\n# Concatenar manteniendo la columna \'ID\'\ntrain_concatenado = pd.concat(train["Sequence"].tolist(), ignore_index=True)\n# Concatenar manteniendo la columna \'ID\'\ntest_concatenado = pd.concat(test["Sequence"].tolist(), ignore_index=True)'

In [ ]:
"""# Dividir los DataFrames en características (X) y etiquetas (y)
x_train = train_concatenado.drop(columns=['Label', 'ID'])
y_train = train_concatenado[['Label', 'ID']]
# Crear la nueva columna basada en el mapeo
y_train['Label_Mapped'] = y_train['Label'].map(LABELS)

x_test = test_concatenado.drop(columns=['Label', 'ID'])
y_test = test_concatenado[['Label', 'ID']]
# Crear la nueva columna basada en el mapeo
y_test['Label_Mapped'] = y_test['Label'].map(LABELS)

print(f"Dimensiones de x_train: {x_train.shape}")
print(y_train[['Label']].value_counts()) 
y_train.head() # Muestra los primeros 5 paquetes del dataframe de entrenamiento"""

'# Dividir los DataFrames en características (X) y etiquetas (y)\nx_train = train_concatenado.drop(columns=[\'Label\', \'ID\'])\ny_train = train_concatenado[[\'Label\', \'ID\']]\n# Crear la nueva columna basada en el mapeo\ny_train[\'Label_Mapped\'] = y_train[\'Label\'].map(LABELS)\n\nx_test = test_concatenado.drop(columns=[\'Label\', \'ID\'])\ny_test = test_concatenado[[\'Label\', \'ID\']]\n# Crear la nueva columna basada en el mapeo\ny_test[\'Label_Mapped\'] = y_test[\'Label\'].map(LABELS)\n\nprint(f"Dimensiones de x_train: {x_train.shape}")\nprint(y_train[[\'Label\']].value_counts()) \ny_train.head() # Muestra los primeros 5 paquetes del dataframe de entrenamiento'

In [ ]:
# Cambiar etiquetas a su estado original
"""original_labels = le.inverse_transform(sequences[-1]['Label'])
print(original_labels)
print(len(original_labels))"""

"original_labels = le.inverse_transform(sequences[-1]['Label'])\nprint(original_labels)\nprint(len(original_labels))"